<a href="https://colab.research.google.com/github/nureeee/DeepLearning/blob/main/%ED%81%B4%EB%9E%98%EC%8A%A4%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EB%84%A4%ED%81%AC%EC%9B%8C%ED%81%AC_%EA%B5%AC%EC%A1%B0_%EC%A0%95%EC%9D%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 사용자 정의 모델링
* tf.keras.model 클래스를 상속 받아서 모델을 구현할 수 있다.
  * 생성자  __init__ 함수에서는 사용할 레이어를 정의합니다.
  * call() 함수 에서는 __init__에서 정의한 레이어를 엮어줍니다.

In [1]:
import tensorflow as tf

In [24]:
class MyModel(tf.keras.Model):
    ## __init__에서는 사용할 레이어들을 정의만 한다
    def __init__(self):
        super(MyModel, self).__init__() # 현재 클래스에 대한 내용을 부모클래스에 넣는다.
        # 부모클래스를 호출하면 강제로 부모클래스의 객체가 생성된다??
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='relu')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    # call에서는 __init__에서 정의한 레이어를 엮어준다. 결과를 반드시 리턴해야된다
    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        out = self.d2(x)
        return out
    

# 학습 루프 정의하기

In [33]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    
    
    
    
    with tf.GradientTape() as tape:
        prediction = model(images)
        loss = loss_object(labels, prediction)

    grads = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, prediction)


# 테스트 루프 정의하기

In [34]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuarcy):
    prediction = model(images)

    t_loss = loss_object(labels, prediction)

    test_loss(t_loss)
    test_accuarcy(labels, prediction)

# 데이터 세트 로딩 및 전처리

In [35]:
mnist = tf.keras.datasets.mnist

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train, X_test = X_train / 255.0 , X_test / 255.0

X_train = X_train[..., tf.newaxis]
X_test = X_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)

# 모델 생성 (네트워크 생성)

In [36]:
model = MyModel()

# 손실함수, 최적화 선택하기

In [37]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [38]:
#

In [39]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 학습 진행

In [42]:
EPOCHS = 10
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)

    template = "Epoch {}, Train Loss : {:.3f}, Train Accuracy : {:.3f} /// Test Loss : {:.3f}, Test Accuracy : {:.3f}"
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100))

Epoch 1, Train Loss : 0.149, Train Accuracy : 95.668 /// Test Loss : 0.063, Test Accuracy : 97.980
Epoch 2, Train Loss : 0.126, Train Accuracy : 96.325 /// Test Loss : 0.057, Test Accuracy : 98.189
Epoch 3, Train Loss : 0.110, Train Accuracy : 96.796 /// Test Loss : 0.051, Test Accuracy : 98.388
Epoch 4, Train Loss : 0.097, Train Accuracy : 97.152 /// Test Loss : 0.047, Test Accuracy : 98.478
Epoch 5, Train Loss : 0.087, Train Accuracy : 97.441 /// Test Loss : 0.043, Test Accuracy : 98.627
Epoch 6, Train Loss : 0.079, Train Accuracy : 97.680 /// Test Loss : 0.039, Test Accuracy : 98.729
Epoch 7, Train Loss : 0.072, Train Accuracy : 97.874 /// Test Loss : 0.037, Test Accuracy : 98.804
Epoch 8, Train Loss : 0.066, Train Accuracy : 98.043 /// Test Loss : 0.034, Test Accuracy : 98.895
Epoch 9, Train Loss : 0.061, Train Accuracy : 98.184 /// Test Loss : 0.032, Test Accuracy : 98.966
Epoch 10, Train Loss : 0.057, Train Accuracy : 98.309 /// Test Loss : 0.030, Test Accuracy : 99.016
